In [31]:
import cv2
import numpy as np
import os

def process_pothole_image(img_path, scale_factor=1.0, output_dir=None):
    try:
        image = cv2.imread(img_path)
        if image is None:
            return None

        # 1. Enhanced Preprocessing
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Dynamic Gaussian Blur based on image size
        img_h, img_w = gray.shape
        kernel_size = 5#max(3, int(min(img_h, img_w) * 0.01) | 1)
        blur = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 2)

        # 2. Canny Edge Detection with adaptive thresholds
        median = np.median(blur)
        lower = int(max(0, 0.6 * median))
        upper = int(min(255, 1.4 * median))
        edges = cv2.Canny(blur, lower, upper)

        # 3. Morphological processing to enhance edges
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        dilated = cv2.dilate(edges, kernel, iterations=2)
        closed = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)

        # 4. Dynamic Parameter Adjustment
        img_area = img_h * img_w
        min_area = (img_area // 150) #* scale_factor  # Adjusted baseline
        max_area = img_area // 3

        # 5. Contour processing with hierarchy analysis
        contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for i, cnt in enumerate(contours):
            if hierarchy[0][i][3] == -1:  # Filter parent contours only
                area = cv2.contourArea(cnt)
                if min_area < area < max_area:
                    x, y, w, h = cv2.boundingRect(cnt)
                    aspect_ratio = w / float(h)
                    if 0.2 < aspect_ratio < 5:  # Filter by shape
                        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Save results if output directory specified
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, os.path.basename(img_path))
            cv2.imwrite(output_path, image)

        return image

    except Exception as e:
        print(f"Error processing {img_path}: {str(e)}")
        return None

# Main processing with configurable parameters
def process_batch(dataset_path, start_idx=0, end_idx=5, scale_factor=1.0, output_dir=None):
    image_files = sorted([f for f in os.listdir(dataset_path) 
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    
    if not image_files:
        print("No images found in dataset directory")
        return

    processed_count = 0
    for img_file in image_files[start_idx:end_idx+1]:
        img_path = os.path.join(dataset_path, img_file)
        result = process_pothole_image(img_path, scale_factor, output_dir)
        
        if result is not None:
            processed_count += 1
            cv2.imshow(f'Detection: {img_file}', result)
            cv2.waitKey(0)  # Brief display for verification

    cv2.destroyAllWindows()
    print(f"Successfully processed {processed_count}/{end_idx-start_idx+1} images")

# Example usage
if __name__ == "__main__":
    dataset_path = "./DS1/pothole_image_data/Pothole_Image_Data"
    process_batch(
        dataset_path=dataset_path,
        start_idx=0,
        end_idx=30,
        scale_factor=1.2,
        output_dir="canny_processed"
    )


Successfully processed 31/31 images
